### A BERTopic-ról
https://maartengr.github.io/BERTopic/algorithm/algorithm.html#6-optional-fine-tune-topic-representation

https://towardsdatascience.com/topics-per-class-using-bertopic-252314f2640

https://people.inf.elte.hu/csa/html/szinek.htm

In [5]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from sentence_transformers import SentenceTransformer

import pickle
import pandas as pd
import huspacy


In [6]:
meta = pickle.load(open("../resources/meta.pkl", "rb"))
lemmatized = pickle.load(open("../resources/lemmatized.pkl", "rb"))
#pos = pickle.load(open("../resources/pos.pkl", "rb"))
#tokens =  pickle.load(open("../resources/tokenized.pkl", "rb"))
#doc_stop = pickle.load(open("../resources/no_stopword.pkl", "rb")) ### kevesebb stopszóval
doc_stop_2 = pickle.load(open("../resources/stopword_filtered.pkl", "rb")) ### több stop szóval, a no_stopword.pkl kiegészítve
all_docs = pickle.load(open("../resources/docs.pkl", "rb"))

## 1. Betöltjük a modellt és a topik infot

In [7]:
#loaded_model = BERTopic.load("../models/bert_probabilities_merged.pkl")

In [8]:
t = pd.read_csv("../results/bert_probabilities_merged_85_new_names.csv",sep=",")
t

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,0,8868,0_placeholder_aha_nem_ja,diskurzuselem,"['placeholder', 'aha', 'nem', 'ja', 'tud', 'mo...","['PlaceHolder', 'PlaceHolder', 'aha ja ja ja']"
1,1,4213,1_hum_igen_ja_persze,"helyeslés, hümmögés","['hum', 'igen', 'ja', 'persze', 'aha', 'mond',...","['hum igen', 'igen hum', 'hum < igen']"
2,2,3033,2_laugh_igen_nem_ja,nevetés,"['laugh', 'igen', 'nem', 'ja', 'tud', 'hum', '...","['laugh', 'laugh', 'laugh']"
3,3,1409,3_tanul_nyelv_angol_egyetem,"tanul, nyelv, angol, egyetem, nem","['tanul', 'nyelv', 'angol', 'egyetem', 'nem', ...","['iskola nem annyira tanul angol', 'angol tanu..."
4,4,1291,4_óra_idő_nem_tud,"óra, idő, nem, tud, nap","['óra', 'idő', 'nem', 'tud', 'nap', 'hogyha', ...",['nap bemegy nyolc fél hét óra óra óra óra vál...
5,5,562,5_barát_ismerkedés_ember_egymás,"barát, ismerkedés, ember, egymás, kapcsolat","['barát', 'ismerkedés', 'ember', 'egymás', 'ka...","['barát', 'barát', 'barát szerinte']"
6,6,436,6_olvas_könyv_elolvas_szeret,"olvas, könyv, elolvas, szeret, film","['olvas', 'könyv', 'elolvas', 'szeret', 'film'...",['igen hogyha könyv kezd végig szeret olvas sz...
7,7,413,7_magyarország_magyar_olaszország_képzel,"magyarorsz, magyar, olaszorszá, képzel, olasz","['magyarország', 'magyar', 'olaszország', 'kép...",['hosszú táv magyarország magyarország képzel ...
8,8,390,8_sportol_sport_foci_edzés,"sportol, sport, foci, edzés, nem","['sportol', 'sport', 'foci', 'edzés', 'nem', '...","['ja sportol', 'ja sportol szeret', 'nem nem s..."
9,9,379,9_éves_család_gyerek_nem,"éves, család, gyerek, nem, szülő","['éves', 'család', 'gyerek', 'nem', 'szülő', '...","['család', 'család nem', 'gyerek család dolog']"


## 2. Megcsináljuk a html-t

In [9]:
html_start = """<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1"  charset="UTF-8">
<style>
div.sticky {
  position: sticky; top:30%;
  background-color: white;
  padding: 50px;
  font-size: 10px; float: right; display:table;
}
table {
    border-collapse: collapse;
    width: 100%;
}
th, td {
    border: 1px solid #7a3f3f;
    padding: 10px;
    text-align: center;
}
</style>
</head>
<body>

<div class="sticky">
    <table>
        <tr>
            <td>Téma</td>
            <td>Szín</td>
        </tr>"""

In [10]:
legend_end = """
</table>
</div>
"""

In [11]:
html_end = """
</body>
</html>
"""

## 3. Összerakjuk az eredeti dialógusokat és a beszédfordulókhoz rendeljük a legvalószínűbb (első helyen álló) topikját, megszínezzük. Egy mappába lementjük az összes html file-t

In [12]:
import os

#main_topics = loaded_model.topics_
main_topics = list(t["Topic"])
with open("../resources/colors.txt", "r") as infile:
    colors = infile.read().strip().split("\n")
ddir = "../data/combined_transcripts_freeConv"
input_files = [f for f in os.listdir(ddir) if os.path.isfile(os.path.join(ddir, f))]
meta2idx = {v: k for k,v in meta.items()}
topic_num2name = dict(zip(t["Topic"], t["CustomName"]))

for input_file in input_files:
    legend_table = []
    local_html = html_start
    # html eleje
    with open(os.path.join(ddir, input_file)) as infile:
        txt = infile.read()
    # találd ki hogy mordor vagy gondor az első
    dialogue_docs = [d for d in meta.values() if d.startswith(input_file)] # a dialógushoz tartozó dokumentumok listája
    dialogue_idx = [meta2idx[d] for d in dialogue_docs] # a dialógushoz tartozó dokumentumok indexeinek listája

    mordor = [d for d in dialogue_docs if "mordor" in d] # a Mordor nevű karakterhez tartozó dokumentumok listája
    mordor_idx = [meta2idx[d] for d in mordor] # a Mordor nevű karakterhez tartozó dokumentumok indexeinek listája
    mordor_topics = [main_topics[i] for i in mordor_idx] # a Mordor nevű karakterhez tartozó dokumentumok témáinak listája
    gondor = [d for d in dialogue_docs if "gondor" in d] # a Gondor nevű karakterhez tartozó dokumentumok listája
    gondor_idx = [meta2idx[d] for d in gondor] # a Gondor nevű karakterhez tartozó dokumentumok indexeinek listája
    gondor_topics = [main_topics[i] for i in gondor_idx] # a Gondor nevű karakterhez tartozó dokumentumok témáinak listája

    if txt.startswith("Mordor:"):
        zipped_idx = list(zip(mordor_idx, gondor_idx))
    else:
        zipped_idx = list(zip(gondor_idx, mordor_idx))

    number_of_topics = list(set(gondor_topics + mordor_topics)) # a dialógusban szereplő témák számának meghatározása
    topic2color = {i: colors[i] for i in number_of_topics}
    for k, v in topic2color.items():
        row = f"""<tr>
                <td>{topic_num2name[k]}</td>
                <td style="background-color: {v}"></td>
            </tr>\n"""
        local_html += row
    local_html += legend_end
    for e in zipped_idx:
        turn1 = all_docs[e[0]]
        turn2 = all_docs[e[1]]
        topic1 = main_topics[e[0]]
        topic2 = main_topics[e[1]]

    o = []
    for e in zipped_idx:
        turn1 = all_docs[e[0]].replace("<", "").replace(">", "")
        turn2 = all_docs[e[1]].replace("<", "").replace(">", "")

        topic1 = main_topics[e[0]]
        topic2 = main_topics[e[1]]

        html1 = f"<p style=\"background-color:{topic2color[topic1]};\">{turn1}</p>"
        html2 = f"<p style=\"background-color:{topic2color[topic2]};\">{turn2}</p>"
        o.append(html1)
        o.append(html2)
    local_html += "\n".join(o)
    final_html = local_html + html_end
    with open(f"../vizs/colored_text/{input_file}.html", "w") as outfile:
        outfile.write(final_html)

IndexError: list index out of range